In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("BlackFriday.csv")

In [3]:
data.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,1422
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,NaN,1057
4,1000002,P00285442,M,55+,16,C,4+,0,8,NaN,NaN,7969


In [5]:
data.shape

(537577, 12)

In [6]:
data.describe()

,User_ID,Occupation,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
count,5.375770e+05,537577.00000,537577.000000,537577.000000,370591.000000,164278.000000,537577.000000
mean,1.002992e+06,8.08271,0.408797,5.295546,9.842144,12.669840,9333.859853
std,1.714393e+03,6.52412,0.491612,3.750701,5.087259,4.124341,4981.022133
min,1.000001e+06,0.00000,0.000000,1.000000,2.000000,3.000000,185.000000
25%,1.001495e+06,2.00000,0.000000,1.000000,5.000000,9.000000,5866.000000
50%,1.003031e+06,7.00000,0.000000,5.000000,9.000000,14.000000,8062.000000
75%,1.004417e+06,14.00000,1.000000,8.000000,15.000000,16.000000,12073.000000
max,1.006040e+06,20.00000,1.000000,18.000000,18.000000,18.000000,23961.000000


In [13]:
total_miss = data.isnull().sum()
percent_miss = total_miss/data.isnull().count()*100
missing_data = pd.DataFrame({'Total missing':total_miss , 'percentage missing':percent_miss})
missing_data.sort_values(by='Total missing',ascending=False).head(3)

,Total missing,percentage missing
Product_Category_3,373299,69.441029
Product_Category_2,166986,31.062713
User_ID,0,0.000000


In [14]:
for i in data.columns:
    print(i,':',data[i].nunique())

User_ID : 5891
Product_ID : 3623
Gender : 2
Age : 7
Occupation : 21
City_Category : 3
Stay_In_Current_City_Years : 5
Marital_Status : 2
Product_Category_1 : 18
Product_Category_2 : 17
Product_Category_3 : 15
Purchase : 17959


In [17]:
males = data[data['Gender']=='M'].count()[0]
females = data[data['Gender']=='F'].count()[0]
print('Number of males are:',males,'\nNumber of females are:',females)

Number of males are: 405380 
Number of females are: 132197


In [20]:
pur_f = data[data['Gender']=='F']['Purchase'].sum()/females
pur_m = data[data['Gender']=='M']['Purchase'].sum()/males
print("female Purchases are:",round(pur_f,3),"\nMale purchases are:",round(pur_m,3))

female Purchases are: 8809.761 
Male purchases are: 9504.772


In [28]:
gender_male = data[data['Gender']=='M'][['Product_Category_1','Gender']].groupby(by='Product_Category_1').count()
gender_female = data[data['Gender']=='F'][['Product_Category_1','Gender']].groupby(by='Product_Category_1').count()
cat_gender = pd.concat([gender_male ,gender_female],axis=1)
cat_gender.columns = ['male ratio','Female Ratio']
cat_gender['male ratio'] = cat_gender['male ratio']/males
cat_gender['Female Ratio'] = cat_gender['Female Ratio']/females
cat_gender['likelihood(M/F)'] = cat_gender['male ratio']/cat_gender['Female Ratio']
cat_gender['total ratio'] = cat_gender['male ratio']+cat_gender['Female Ratio']
cat_gender.sort_values(by='likelihood(M/F)',ascending=False)

,male ratio,Female Ratio,likelihood(M/F),total ratio
Product_Category_1,,,,
17,0.001248,0.000461,2.705079,0.001710
18,0.006658,0.002844,2.340854,0.009502
15,0.012778,0.007738,1.651252,0.020517
9,0.000824,0.000530,1.555993,0.001353
1,0.281099,0.184581,1.522908,0.465680
11,0.047612,0.035243,1.350972,0.082855
6,0.038702,0.033851,1.143303,0.072553
10,0.009606,0.008608,1.115868,0.018214
2,0.044220,0.042157,1.048947,0.086377


In [31]:
data['Age'].unique()

array(['0-17', '55+', '26-35', '46-50', '51-55', '36-45', '18-25'],
      dtype=object)

In [32]:
data['Age Encoded'] = data['Age'].map({'0-17':0, '55+':6, '26-35':2, '46-50':4, '51-55':5, '36-45':3, '18-25':1})

In [36]:
pro_byage = data.groupby('Age').nunique()['Product_ID']

In [37]:
pro_byage

Age
0-17     2300
18-25    3213
26-35    3419
36-45    3318
46-50    3099
51-55    2877
55+      2573
Name: Product_ID, dtype: int64